# Parking mandates

## Hypothesis

The off-street parking mandate may be requiring more space dedicated to parking than is necessary.

In [9]:
from dvc.api import DVCFileSystem
import pandas as pd

fs = DVCFileSystem("https://github.com/tjrileywisc/waltham_etl")

## Fetch American Community Survey data for Waltham census tracts

In [10]:
with fs.open("data/census/data_profiles_census_df.csv") as f:
    
    to_keep = ["DP04_0057E", "DP04_0058E", "DP04_0059E", "DP04_0060E", "DP04_0061E"]
    
    data_profiles_df = pd.read_csv(f, usecols=to_keep, dtype={col: int for col in to_keep})
    
    data_profiles_df.index.name = "tract"
    
    data_profiles_df = data_profiles_df[to_keep]
    
    print(data_profiles_df)

       DP04_0057E  DP04_0058E  DP04_0059E  DP04_0060E  DP04_0061E
tract                                                            
0            2151         142         850         935         224
1            1926          61         566         664         635
2            1414          40         537         584         253
3            2390         269         818         949         354
4            2220          71         920        1080         149
5            1709         279         988         368          74
6            2323         148        1197         750         228
7            1092          57         526         474          35
8            2431         262        1067         783         319
9            2812          68        1233        1264         247
10           1353         120         624         583          26
11             20           0           9          11           0
12           2050         100         608         992         350


In [11]:
with fs.open("data/census/subject_census_df.csv") as f:
    subject_df = pd.read_csv(f, index_col=0, dtype={"S0101_C01_026E": int})
    
    subject_df.index.name = "tract"
    
    print(subject_df)

         S0101_C01_026E
tract                  
3681.01            4361
3681.02            4690
3682.00            3023
3683.00            4346
3684.00            7091
3685.00            2716
3686.00            4674
3687.00            2310
3688.00            4549
3689.01            5129
3689.02            2689
3690.00            4062
3691.00            4402


## Calculate number of adults per car

The usual assumption is that every adult in a household unit 'needs' a car, and any new construction should provide that amount of parking,
so incoming residents do not use on-street parking for vehicle storage.

This is set as ['2 per dwelling unit'](https://ecode360.com/26938091) for family dwellings.

This is assuming the typical 'nuclear' family with two adults who are doing the driving. If all adults are driving, one would expect that the number
of adults = the number of cars, so this ratio should come out to 1.

I have yet to find data that gets the number of cars in a census tract, so I have decided to calculate this from the survey questions as follows:

`n_cars = 1 * households with one car + 2 * households with two cars + 3 * households with 3 or more cars`

*Assumption* - in most cases, the number of cars in the '3 or more category' is going to tend to be closer to 3 than other values, for most homes.
If this assumption is wrong, `n_cars` will be lower than the actual amount.

In [12]:
# Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 years and over
total_adults = subject_df["S0101_C01_026E"]

# # Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!No vehicles available
# "DP04_0058E",
# # Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!1 vehicle available
# "DP04_0059E",
# # Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!2 vehicles available
# "DP04_0060E",
# # Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available
# "DP04_0061E"
parking_mandates_df = pd.DataFrame(columns=["total_vehicles", "total_adults", "adults_per_vehicle"])

parking_mandates_df["total_adults"] = subject_df["S0101_C01_026E"]


# ASSUMPTION - more than 3 vehicles will be rare, so assume 3 per household unit for that bucket
parking_mandates_df["total_vehicles"] = (1 * data_profiles_df["DP04_0059E"] + 2 * data_profiles_df["DP04_0060E"] + 3 * data_profiles_df["DP04_0061E"]).values

parking_mandates_df["adults_per_vehicle"] = parking_mandates_df["total_adults"] / parking_mandates_df["total_vehicles"]

print(parking_mandates_df)

         total_vehicles  total_adults  adults_per_vehicle
tract                                                    
3681.01            3392          4361            1.285672
3681.02            3799          4690            1.234535
3682.00            2464          3023            1.226867
3683.00            3778          4346            1.150344
3684.00            3527          7091            2.010491
3685.00            1946          2716            1.395683
3686.00            3381          4674            1.382431
3687.00            1579          2310            1.462951
3688.00            3590          4549            1.267131
3689.01            4502          5129            1.139271
3689.02            1868          2689            1.439507
3690.00              31          4062          131.032258
3691.00            3642          4402            1.208677


## Comments

Most tracts come in at more than one adult per vehicle, except for tract 3690 and 3684.

3690 is mostly Bentley University, so we would expect many adults who are non-driving students.

3684 mostly overlaps ward 7, which is dominated by single family homes. It appears that many adults are sharing vehicles, but
this tract also contains Brandeis University. 

*Assumption* - It's not possible to tell if the all of the [3,031 18-24 year olds in tertiary eduation](https://data.census.gov/map?g=0400000US25_1400000US25017368101,25017368102,25017368200,25017368300,25017368400,25017368500,25017368600,25017368700,25017368800,25017368901,25017368902,25017369000,25017369100&tid=ACSST5Y2020.S1401&cid=S1401_C01_030E&layer=VT_2020_140_00_PY_D1&mode=thematic&loc=42.3818,-71.2223,z12.3362) in this tract are going to Brandeis, but assuming they are, the ratio would come out to

`(7091 - 3031) / 3527 = ~1.15 adults / vehicle`

## Source

This data comes from the 5-year American Community Survey estimate from 2020. If anything, I would assume that the number
of vehicles would drop somewhat due to more adults no longer needing to commute to an office.

You can look at the uncombined data on the tracts [here](https://data.census.gov/map?q=DP04&g=1400000US25017368101,25017368102,25017368200,25017368300,25017368400,25017368500,25017368600,25017368700,25017368800,25017368901,25017368902,25017369000,25017369100&tid=ACSDP5Y2020.DP04&cid=DP04_0060PE&layer=VT_2020_140_00_PY_D1&mode=thematic&loc=42.3628,-71.2427,z13.6751). Change the `Variable(s)` field
to see how each tract fits in each category.